In [1]:
import pandas as pd
import Levenshtein
import itertools
from fuzzywuzzy import fuzz
import ast
from selenium import webdriver
import time
import webbrowser

In [2]:
recipe_df = pd.read_csv('../data/processed/recipe_df.csv')

pd.set_option('max_columns', None)

recipe_df.head()

,recipe_image_url,recipe_name,recipe_url,ingredients_list,time_preparation(min),calories(kcal),gluten_free,egg_free,sucrose_fructose_free,low_sodium,lactose_free,tipo_plato,main_category,category,num_ingredients
0,https://content-recetas.lecturas.com/medio/201...,Ensalada niçoise,https://www.lecturas.com/recetas/ensalada-nico...,"['patatas pequeñas rojas', 'huevos', 'judías v...",60.0,252.0,0,0,0,0,0,['Primeros platos y entrantes'],principal,ensaladas,12
1,https://content-recetas.lecturas.com/medio/201...,Ensalada del périgord,https://www.lecturas.com/recetas/ensalada-peri...,"['foie micuit', 'beicon gruesas', 'champiñones...",25.0,386.0,0,0,0,0,0,['Primeros platos y entrantes'],principal,ensaladas,9
2,https://content-recetas.lecturas.com/medio/202...,Xató con salsa romesco,https://www.lecturas.com/recetas/xato-salsa-ro...,"['bacalao desalado', 'cebolleta', 'escarolas',...",35.0,NaN,0,0,0,0,0,['Primeros platos y entrantes'],principal,ensaladas,16
3,https://content-recetas.lecturas.com/medio/201...,Ensalada de ceviche de langostinos y cebolla,https://www.lecturas.com/recetas/ensalada-cevi...,"['langostinos', 'pimienta', 'limas', 'tomates'...",25.0,176.0,0,0,0,0,0,['Primeros platos y entrantes'],principal,ensaladas,12
4,https://content-recetas.lecturas.com/medio/201...,"Ensalada de pollo, remolacha y manzana",https://www.lecturas.com/recetas/ensalada-poll...,"['pechugas de pollo', 'manzanas ácida', 'col l...",40.0,203.0,0,0,0,0,0,['Primeros platos y entrantes'],principal,ensaladas,16


In [3]:
#recipe_df["ingredient"].value_counts()

In [4]:
input_ing = input('¿Qué tienes hoy en la nevera?')

¿Qué tienes hoy en la nevera? huevos, pechiga de pollo, lentejas, pimiento, arroz, leche, mantequilla, harina, sal, azúcar, aceite, zanahorias, queso, bacon


In [5]:
input_ing

'huevos, pechiga de pollo, lentejas, pimiento, arroz, leche, mantequilla, harina, sal, azúcar, aceite, zanahorias, queso, bacon'

In [23]:
def input_list(string):
    input_list = []
    str_split = string.split(", ")
    for x in str_split:
        input_list.append(x)
    return input_list

input_ingred = input_list(input_ing)

print(input_ingred)

['huevos', 'pechiga de pollo', 'lentejas', 'pimiento', 'arroz', 'leche', 'mantequilla', 'harina', 'sal', 'azúcar', 'aceite', 'zanahorias', 'queso', 'bacon']


In [7]:
def fuzzy_columns(column, input_string):
    ingred_list = ast.literal_eval(column)
    for ingred in ingred_list:
        dif = fuzz.token_set_ratio(column, input_string)
    if dif > 70:
        return True
    else:
        return False


In [8]:
for ingred in input_ingred:
    recipe_df[ingred] = recipe_df.apply(lambda x: fuzzy_columns(x["ingredients_list"], ingred), axis=1)

In [21]:
recipe_df.head(1)

,recipe_image_url,recipe_name,recipe_url,ingredients_list,time_preparation(min),calories(kcal),gluten_free,egg_free,sucrose_fructose_free,low_sodium,lactose_free,tipo_plato,main_category,category,num_ingredients,huevos,pechiga de pollo,lentejas,pimiento,arroz,leche,mantequilla,harina,sal,azúcar,aceite,zanahorias,queso,bacon,active_ingred,num_missing_ingred,recipe_rate
0,https://content-recetas.lecturas.com/medio/201...,Ensalada niçoise,https://www.lecturas.com/recetas/ensalada-nico...,"['patatas pequeñas rojas', 'huevos', 'judías v...",60.0,252.0,0,0,0,0,0,['Primeros platos y entrantes'],principal,ensaladas,12,True,False,False,False,False,False,False,False,True,False,True,False,False,False,3,9,0.0


In [10]:
recipe_df["active_ingred"] = recipe_df.select_dtypes(include=['bool']).sum(axis=1)

In [11]:
def num_missing_ingred(recipe_ingred, active_ingred):
    missing_ingred = recipe_ingred - active_ingred
    return missing_ingred

recipe_df["num_missing_ingred"] = recipe_df.apply(lambda x: num_missing_ingred(x["num_ingredients"], x["active_ingred"]), axis=1)

In [12]:
recipe_df.head(3)

,recipe_image_url,recipe_name,recipe_url,ingredients_list,time_preparation(min),calories(kcal),gluten_free,egg_free,sucrose_fructose_free,low_sodium,lactose_free,tipo_plato,main_category,category,num_ingredients,huevos,pechiga de pollo,lentejas,pimiento,arroz,leche,mantequilla,harina,sal,azúcar,aceite,zanahorias,queso,bacon,active_ingred,num_missing_ingred
0,https://content-recetas.lecturas.com/medio/201...,Ensalada niçoise,https://www.lecturas.com/recetas/ensalada-nico...,"['patatas pequeñas rojas', 'huevos', 'judías v...",60.0,252.0,0,0,0,0,0,['Primeros platos y entrantes'],principal,ensaladas,12,True,False,False,False,False,False,False,False,True,False,True,False,False,False,3,9
1,https://content-recetas.lecturas.com/medio/201...,Ensalada del périgord,https://www.lecturas.com/recetas/ensalada-peri...,"['foie micuit', 'beicon gruesas', 'champiñones...",25.0,386.0,0,0,0,0,0,['Primeros platos y entrantes'],principal,ensaladas,9,False,False,False,False,False,False,False,False,False,False,True,False,False,False,1,8
2,https://content-recetas.lecturas.com/medio/202...,Xató con salsa romesco,https://www.lecturas.com/recetas/xato-salsa-ro...,"['bacalao desalado', 'cebolleta', 'escarolas',...",35.0,NaN,0,0,0,0,0,['Primeros platos y entrantes'],principal,ensaladas,16,False,False,False,False,False,False,False,False,True,False,True,False,False,False,2,14


In [13]:
def rating(match, missing):
    result = 0.75*match-0.25*missing
    return result

recipe_df["recipe_rate"] = recipe_df.apply(lambda x: rating(x["active_ingred"], x["num_missing_ingred"]), axis=1)

In [14]:
recipe_df.head(3)

,recipe_image_url,recipe_name,recipe_url,ingredients_list,time_preparation(min),calories(kcal),gluten_free,egg_free,sucrose_fructose_free,low_sodium,lactose_free,tipo_plato,main_category,category,num_ingredients,huevos,pechiga de pollo,lentejas,pimiento,arroz,leche,mantequilla,harina,sal,azúcar,aceite,zanahorias,queso,bacon,active_ingred,num_missing_ingred,recipe_rate
0,https://content-recetas.lecturas.com/medio/201...,Ensalada niçoise,https://www.lecturas.com/recetas/ensalada-nico...,"['patatas pequeñas rojas', 'huevos', 'judías v...",60.0,252.0,0,0,0,0,0,['Primeros platos y entrantes'],principal,ensaladas,12,True,False,False,False,False,False,False,False,True,False,True,False,False,False,3,9,0.00
1,https://content-recetas.lecturas.com/medio/201...,Ensalada del périgord,https://www.lecturas.com/recetas/ensalada-peri...,"['foie micuit', 'beicon gruesas', 'champiñones...",25.0,386.0,0,0,0,0,0,['Primeros platos y entrantes'],principal,ensaladas,9,False,False,False,False,False,False,False,False,False,False,True,False,False,False,1,8,-1.25
2,https://content-recetas.lecturas.com/medio/202...,Xató con salsa romesco,https://www.lecturas.com/recetas/xato-salsa-ro...,"['bacalao desalado', 'cebolleta', 'escarolas',...",35.0,NaN,0,0,0,0,0,['Primeros platos y entrantes'],principal,ensaladas,16,False,False,False,False,False,False,False,False,True,False,True,False,False,False,2,14,-2.00


In [15]:
filter_principal = recipe_df['main_category'] == "principal"

output_df = recipe_df[filter_principal].sort_values(by=['recipe_rate', 'num_missing_ingred'], ascending=[False, True]).head(10)

output_df

,recipe_image_url,recipe_name,recipe_url,ingredients_list,time_preparation(min),calories(kcal),gluten_free,egg_free,sucrose_fructose_free,low_sodium,lactose_free,tipo_plato,main_category,category,num_ingredients,huevos,pechiga de pollo,lentejas,pimiento,arroz,leche,mantequilla,harina,sal,azúcar,aceite,zanahorias,queso,bacon,active_ingred,num_missing_ingred,recipe_rate
2890,https://content-recetas.lecturas.com/medio/201...,Crepes de espinacas y queso de cabra,https://www.lecturas.com/recetas/crepes-espina...,"['espinacas', 'diente de ajo', 'queso de cabra...",30.0,425.0,0,0,0,0,0,['Primeros platos y entrantes'],principal,"quiche, creps y pasteles salados",10,False,False,False,False,False,True,True,True,True,True,True,False,True,False,7,3,4.50
2973,https://content-recetas.lecturas.com/medio/201...,Pastelitos de espárragos trigueros,https://www.lecturas.com/recetas/pastelitos-es...,"['espárrago triguero', 'cebolla', 'mantequilla...",40.0,285.0,0,0,0,0,0,['Primeros platos y entrantes'],principal,"quiche, creps y pasteles salados",11,True,False,False,False,False,True,True,True,True,False,True,False,True,False,7,4,4.25
2980,https://content-recetas.lecturas.com/medio/201...,Tarta ligera de coliflor y queso,https://www.lecturas.com/recetas/tarta-ligera-...,"['coliflores', 'huevos', 'cebolla', 'pimiento ...",80.0,158.0,0,0,0,0,0,"['Primeros platos y entrantes', 'Segundos plat...",principal,"quiche, creps y pasteles salados",11,True,False,False,True,False,True,False,True,True,False,True,False,True,False,7,4,4.25
2150,https://content-recetas.lecturas.com/medio/201...,Volován de pollo y mozzarella,https://www.lecturas.com/recetas/volovan-pollo...,"['volovanes', 'pechugas de pollo', 'huevos', '...",35.0,290.0,0,0,0,0,0,['Segundos platos y platos principales'],principal,aves,12,True,False,False,False,False,True,True,True,True,False,True,False,True,False,7,5,4.00
2236,"data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP//...",Solomillos de pollo rebozados con patatas,https://www.lecturas.com/recetas/solomillos-po...,"['solomillos de pollo', 'huevos', 'leche', 'li...",60.0,NaN,0,0,0,0,0,['Segundos platos y platos principales'],principal,aves,12,True,False,False,False,False,True,True,True,True,False,True,False,True,False,7,5,4.00
766,https://content-recetas.lecturas.com/medio/201...,Calabaza asada rellena,https://www.lecturas.com/recetas/calabaza-asad...,"['calabaza pequeñas', 'diente de ajo', 'ceboll...",65.0,NaN,0,0,0,0,0,['Primeros platos y entrantes'],principal,verduras,13,False,False,False,True,False,True,False,True,True,True,True,False,True,False,7,6,3.75
803,https://content-recetas.lecturas.com/medio/201...,Calabacines rellenos a los cuatro quesos,https://www.lecturas.com/recetas/calabacines-r...,"['calabacines', 'tomates', 'cebolla', 'zanahor...",60.0,NaN,0,0,0,0,0,['Primeros platos y entrantes'],principal,verduras,13,False,False,False,True,False,True,False,True,True,False,True,True,True,False,7,6,3.75
1161,https://content-recetas.lecturas.com/medio/201...,Lasaña a la hortelana,https://www.lecturas.com/recetas/lasana-a-hort...,"['placas', 'calabacín', 'pimiento rojo', 'cebo...",90.0,NaN,0,0,0,0,0,['Primeros platos y entrantes'],principal,pastas,13,False,False,False,True,False,True,True,True,True,False,True,False,True,False,7,6,3.75
5170,https://content-recetas.lecturas.com/medio/201...,Trenza de pan blanco rellena de pesto,https://www.lecturas.com/recetas/trenza-pan-bl...,"['harina de trigo', 'levadura en polvo seca', ...",85.0,NaN,0,0,0,0,0,['Acompañamientos'],principal,pizzas y empanadas,13,False,False,False,False,False,True,True,True,True,True,True,False,True,False,7,6,3.75
440,https://content-recetas.lecturas.com/medio/201...,"Timbal de calabaza, patata y queso",https://www.lecturas.com/recetas/timbal-calaba...,"['calabaza de unos 600 g', 'patatas', 'calabac...",70.0,345.0,0,0,0,0,0,['Primeros platos y entrantes'],principal,verduras,10,False,False,False,False,False,True,True,True,True,False,True,False,True,False,6,4,3.50


In [16]:
output_df.columns

Index(['recipe_image_url', 'recipe_name', 'recipe_url', 'ingredients_list',
       'time_preparation(min)', 'calories(kcal)', 'gluten_free', 'egg_free',
       'sucrose_fructose_free', 'low_sodium', 'lactose_free', 'tipo_plato',
       'main_category', 'category', 'num_ingredients', 'huevos',
       'pechiga de pollo', 'lentejas', 'pimiento', 'arroz', 'leche',
       'mantequilla', 'harina', 'sal', 'azúcar', 'aceite', 'zanahorias',
       'queso', 'bacon', 'active_ingred', 'num_missing_ingred', 'recipe_rate'],
      dtype='object')

In [17]:
recipe_name = output_df["recipe_name"].iloc[0]

num_missing_ingred = output_df['num_missing_ingred'].iloc[0]

recipe_time = output_df["time_preparation(min)"].iloc[0]

print(f"La mejor receta que puedes hacer con los ingredientes que has introducido es: {recipe_name}")
print(f"Te faltan {num_missing_ingred} ingredientes")
print(f"El tiempo de preparación es de {recipe_time} minutos")

La mejor receta que puedes hacer con los ingredientes que has introducido es: Crepes de espinacas y queso de cabra
Te faltan 3 ingredientes
El tiempo de preparación es de 30.0 minutos


In [18]:
go_to_recipe = input('¿Quieres ir a la receta?')

def normalize(string):
    lower = string.lower()
    a,b = 'áéíóú','aeiou'
    trans = str.maketrans(a,b)
    result = lower.translate(trans)
    return result

input_answer = normalize(go_to_recipe)

¿Quieres ir a la receta? si


In [19]:
if input_answer == 'si':
    recipe_link = output_df["recipe_url"].iloc[0]
    def pop_recipe(url_):
        ROUTE = url_
        webbrowser.open_new(ROUTE)
    pop_recipe(recipe_link)
elif input_answer == 'no':
    recipe_link = output_df["recipe_url"].iloc[0]
    print("Genial! Éste es el link por si quieres consultar la receta más tarde: {recipe_link}")
else:
    print("Por favor, responde sí o no")